### Importing Libraries

In [1]:
import tensorflow as tf
from tqdm import notebook
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

2.3.0
2.4.0


### Hyper Parameters

In [2]:
learning_rate = 0.001
training_epochs = 3
batch_size = 100

tf.random.set_seed(777)

### Creating a Checkpoint Directory

In [3]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'mnist_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

### MNIST/Fashion MNIST Data

In [4]:
# MNIST data
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

### Datasets

In [5]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# 정규화
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

# 채널 추가
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)

# to categorical
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).\
                                                    shuffle(buffer_size=100000).\
                                                    batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).\
                                                   batch(batch_size)

### Model Function

In [6]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3,
                                  activation=tf.nn.relu, padding='SAME',
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3,
                                  activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3,
                                  activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

### Loss Function

In [8]:
@tf.function
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))
    return loss

### Calculating Gradient

In [9]:
@tf.function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.variables)

### Calculating Model's Accuracy

In [10]:
@tf.function
def evaluate(model, images, labels):
    logits = model(images, training=False) # training=True : dropout 사용 유무
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

### Optimizer

In [11]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

### Creating a Checkpoint

In [12]:
checkpoint = tf.train.Checkpoint(cnn=model)

### Training

In [13]:
@tf.function
def train(model, images, labels):
    grads = grad(model, images, labels)
    optimizer.apply_gradients(zip(grads, model.variables))

In [14]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None, 10)), types: (tf.float32, tf.float32)>

In [15]:
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in notebook.tqdm(train_dataset):
        train(model, images, labels)
        loss = loss_func(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss += loss
        avg_train_acc += acc
        train_step += 1
    
    avg_loss /= train_step
    avg_train_acc /= train_step
    
    for images, labels in test_dataset:
        acc = evaluate(model, images, labels)
        avg_test_acc += acc
        test_step += 1
    
    avg_test_acc /= test_step
    
    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)


Epoch: 1 loss = 0.16154166 train accuracy =  0.9593 test accuracy =  0.9850



Epoch: 2 loss = 0.03897562 train accuracy =  0.9905 test accuracy =  0.9910



Epoch: 3 loss = 0.02470518 train accuracy =  0.9937 test accuracy =  0.9920
